<a href="https://colab.research.google.com/github/anahimaciel/Networks-in-cities/blob/main/RadiusCalculatorAuto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import geopy.distance

In [6]:
#2m39
for city in os.listdir("/content/drive/MyDrive/Networks in cities/Dados"):
  if(city=="Fortaleza" or city==".ipynb_checkpoints"):
    continue

  print(city)

  #read df
  df = pd.read_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/filtered_cdr.txt",usecols=[1,2,3,4,5])
  #read antennas files
  map = pd.read_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/antennas_"+city+".txt", sep=';')

  #convert to datetime object
  df['date']=pd.to_datetime(df['date'])
  #create column with days of the week
  df['day_week']=df['date'].dt.day_name()

  #create dataframe with calls in weekends and nights
  night_df=df[(df['day_week']=='Saturday') | (df['day_week']=='Sunday') | (df['time']> '19:00:00') | (df['time']< '06:00:00')]

  #create dataframe with data necessary to compute places of residence
  residence_df=night_df.groupby(['user_from','antenna']).size().reset_index(name='time')

  #compute places of residence
  #drop locations other than the first
  residence_df=residence_df.sort_values(['user_from','time'],ascending=[True,False]).drop_duplicates('user_from')
  #only keep users whose first location complies with certain parameters (more than half of the calls of the user)
  number_calls_map=residence_df['user_from'].map(df.groupby('user_from').size())
  filter=(residence_df['time']>number_calls_map/2)
  residence_df=residence_df[filter]

  #include geographical location of antennas in residence dataframe
  map=map.rename(columns={"CELLID": "antenna"})
  residence_df=pd.merge(residence_df,map,on="antenna",how="left")

  #delete unnecessary columns
  residence_df=residence_df.drop(['time','UF','DDD','CITY','MUNICIPALITY'],axis=1)
  residence_df.sort_values('antenna')

  #extract from df users whose place of residence were not found (useful for next step)
  filter=df['user_from'].isin(residence_df['user_from']) & df['user_to'].isin(residence_df['user_from'])
  filtered_df=df[filter]

  #dicts with coordinates
  dict_lat = residence_df.set_index('user_from')['LAT'].to_dict()
  dict_long = residence_df.set_index('user_from')['LONG'].to_dict()

  #function to calculate distance between residences of caller and receiver for each call
  def get_distance(row):
      coords_caller=(dict_lat[row.user_from],dict_long[row.user_from])                #deixar mais rapido: agrupar por pessoa q ta ligando (n sei se ajudaria mt)
      coords_receiver=(dict_lat[row.user_to],dict_long[row.user_to])
      return geopy.distance.distance(coords_caller,coords_receiver).km

  #apply function to filtered_df
  filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)

  #drop calls to the same receiver (nao sei pq)
  groups=filtered_df.groupby(['user_from','user_to'])
  aux=groups.first()
  unique_calls=len(aux)
  aux=aux.groupby('user_from')['distance'].sum().to_frame(name='total_distance_without_alpha')
  non_zero_df=aux[aux['total_distance_without_alpha']!=0]

  #calculate average radius for each user
  radii=filtered_df.groupby("user_from").agg({'distance':['sum','size'],'user_to':'nunique'})
  radii.columns=['total_distance','n_calls','contacts']
  radii['avg_dist_by_calls']=radii['total_distance']/radii['n_calls']
  radii['avg_radius']=radii['total_distance']/radii['contacts']
  radii['total_distance_without_alpha']=aux['total_distance_without_alpha']
  radii['avg_radius_without_alpha']=aux['total_distance_without_alpha']/radii['contacts']

  #results
  with open("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/results.txt",'a') as file:
    file.truncate(0)
    for i in radii.columns:
      results_df=radii.agg({i:['max','min','mean']})
      file.write(results_df.to_string(index=False)+"\n")

  #saving demography data
  with open("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/demo2.txt",'a') as file:
    file.truncate(0)
    file.write("users with residence: "+str(len(residence_df))+"\n")
    file.write("calls by users with residence: "+str(len(filtered_df))+"\n")
    file.write("calls to unique contacts by users with residence: "+str(len(aux))+"\n")
    file.write("calls to unique contacts by users with residence with distance != 0: "+ str(len(non_zero_df))+"\n")

Franca


<ipython-input-6-b403b9371b71>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)


Jundiaí


<ipython-input-6-b403b9371b71>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)


Piracicaba


...

<ipython-input-6-b403b9371b71>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)


Ponta Grossa


<ipython-input-6-b403b9371b71>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)


Rio Branco


<ipython-input-6-b403b9371b71>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)
